In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../src')

import math
import cadquery as cq
from ocp_vscode import show

from chapel2.dome_generator import generate_dome_with_hubs

from chapel2.geometry import build_vertex_to_edges_map

from chapel2.analysis import (
    full_manufacturability_analysis,
    format_analysis_report,
)

from chapel2.visualization import (
    separate_dome_parts,
    VISUALIZATION_COLORS,
    VISUALIZATION_ALPHAS,
    VISUALIZATION_NAMES,
    STAINED_GLASS_COLORS,
    generate_stained_glass_colors,
)

In [3]:
# Use imported color palette, alphas, and names from visualization module
COLORS = VISUALIZATION_COLORS
ALPHAS = VISUALIZATION_ALPHAS
NAMES = VISUALIZATION_NAMES


# Chapel of MOOP - 8ft 3V Honeycomb Dome

Configuration:
- **Style**: Honeycomb (Hex/Pent)
- **Radius**: 8 ft
- **Frequency**: 3V
- **Hubs**: Tapered Prism
- **Struts**: Cuboid
- **Windows**: Hexagonal/Pentagonal Plates

In [4]:
# Generate Chapel dome with tapered prism hubs
print("Generating Chapel dome with TAPERED PRISM hubs...")

struts_prism, hubs_prism, windows_prism, info_prism = generate_dome_with_hubs(
    radius_cm=8.0 * 30.48,
    frequency=3,
    strut_width=3.5 * 2.54,
    strut_depth=3.5 * 2.54,
    dome_style="honeycomb",
    hub_style="tapered_prism",
    strut_style="cuboid",
    generate_windows=True,
    window_plate_depth=2 * 2.54,
)

print(f"  Struts: {info_prism['num_struts_generated']}")
print(f"  Hubs: {info_prism['num_hubs_generated']}")
print(f"  Windows: {info_prism['num_windows_generated']}")

# Visualize tapered prism dome
print("Chapel Dome with Tapered Prism Hubs and Windows:")
show(struts_prism, hubs_prism, windows_prism)

Generating Chapel dome with TAPERED PRISM hubs...
  Struts: 264
  Hubs: 168
  Windows: 89
Chapel Dome with Tapered Prism Hubs and Windows:
Using port 3939
+++


## Selective Visualization: Complete vs Partial Parts

Generate separate compounds for complete and partial/cut parts at the base.
This allows visualizing:
- Complete dome (all parts above the base)
- Partial dome (only parts cut at the base)
- With or without panes


In [5]:
# Generate separated dome parts (complete vs partial)
print("Generating separated dome parts...")

separated = separate_dome_parts(
    vertices=info_prism['vertices'],
    edges=info_prism['edges'],
    faces=info_prism['faces'],
    radius_cm=info_prism['radius_cm'],
    portion=info_prism['portion'],
    strut_width=info_prism['strut_width'],
    strut_depth=info_prism['strut_depth'],
    hub_inset=info_prism['hub_inset'],
    window_plate_depth=2 * 2.54,  # 2 inches
    window_margin=0.2,
)

print("\nPart counts:")
for part_type, count in separated['counts'].items():
    print(f"  {part_type}: {count}")


Generating separated dome parts...

Part counts:
  complete_struts: 240
  partial_struts: 24
  complete_hubs: 144
  boundary_hubs: 24
  complete_panes: 73
  partial_panes: 16


### View 1: Complete Dome (fully above base) - WITHOUT panes


In [6]:
# View 1: Complete parts only - WITHOUT panes
parts_to_show = []
part_colors = []
part_alphas = []
part_names = []

for key in ['complete_struts', 'complete_hubs', 'boundary_hubs']:
    if separated[key]:
        parts_to_show.append(separated[key])
        part_colors.append(COLORS[key])
        part_alphas.append(ALPHAS[key])
        part_names.append(NAMES[key])

print(f"Complete dome WITHOUT panes:")
print(f"  Struts: {separated['counts']['complete_struts']}")
print(f"  Hubs: {separated['counts']['complete_hubs']+separated['counts']['boundary_hubs']}")
show(*parts_to_show, colors=part_colors, alphas=part_alphas, names=part_names)


Complete dome WITHOUT panes:
  Struts: 240
  Hubs: 168
+++


### View 2: Partial Dome (cut at base) - WITH partial panes


In [7]:
# View 2: All parts including partial/cut - WITH panes
parts_to_show = []
part_colors = []
part_alphas = []
part_names = []

for key in ['complete_struts', 'complete_hubs', 'partial_struts', 'boundary_hubs', 'partial_panes']:
    if separated[key]:
        parts_to_show.append(separated[key])
        part_colors.append(COLORS[key])
        part_alphas.append(ALPHAS[key])
        part_names.append(NAMES[key])

print(f"Partial dome (cut at base) WITH panes:")
print(f"  Struts: {separated['counts']['partial_struts']}")
print(f"  Boundary Hubs: {separated['counts']['boundary_hubs']}")
print(f"  Panes: {separated['counts']['partial_panes']}")
show(*parts_to_show, colors=part_colors, alphas=part_alphas, names=part_names)

Partial dome (cut at base) WITH panes:
  Struts: 24
  Boundary Hubs: 24
  Panes: 16
cc+c+


### View 3: Complete Dome (fully above base) - WITH panes


In [8]:
# View 3: Complete parts only - WITH panes
parts_to_show = []
part_colors = []
part_alphas = []
part_names = []

for key in ['complete_struts', 'complete_hubs', 'complete_panes']:
    if separated[key]:
        parts_to_show.append(separated[key])
        part_colors.append(COLORS[key])
        part_alphas.append(ALPHAS[key])
        part_names.append(NAMES[key])

print(f"Complete dome WITH panes:")
print(f"  Struts: {separated['counts']['complete_struts']}")
print(f"  Hubs: {separated['counts']['complete_hubs']}")
print(f"  Panes: {separated['counts']['complete_panes']}")
show(*parts_to_show, colors=part_colors, alphas=part_alphas, names=part_names)


Complete dome WITH panes:
  Struts: 240
  Hubs: 144
  Panes: 73
cc+


### View 4: Full Dome (cut at base)


In [22]:
# View 4: Full dome with all parts (cut at base)
parts_to_show = []
part_colors = []
part_alphas = []
part_names = []

for key in ['complete_struts', 'complete_hubs', 'partial_struts', 'boundary_hubs', 'complete_panes', 'partial_panes']:
    if separated[key]:
        parts_to_show.append(separated[key])
        part_colors.append(COLORS[key])
        part_alphas.append(ALPHAS[key])
        part_names.append(NAMES[key])

print(f"Full dome with all parts:")
for key, count in separated['counts'].items():
    print(f"  {NAMES[key]}: {count}")
show(*parts_to_show, colors=part_colors, alphas=part_alphas, names=part_names)


Full dome with all parts:
  Complete Struts: 240
  Partial Struts: 24
  Complete Hubs: 144
  Boundary Hubs: 24
  Complete Panes: 73
  Partial Panes: 16
cccccc


### View 5: Stained Glass Mode

Dark brown structure with randomly colored panes (yellow, red, blue, green, purple).


In [17]:
# View 5: Stained Glass Mode - dark brown structure, colorful panes
# Extract individual panes from the complete_panes compound for individual coloring
complete_panes_compound = separated['complete_panes']
pane_solids = list(complete_panes_compound.Solids()) if complete_panes_compound else []

# Generate random colors for each pane
pane_colors = generate_stained_glass_colors(len(pane_solids), seed=20)

# Build parts list: structure (dark brown) + individual panes (random colors)
parts_to_show = []
part_colors = []
part_alphas = []
part_names = []

# Add structural elements (all dark brown)
structure_color = STAINED_GLASS_COLORS['structure']
for key in ['complete_struts', 'complete_hubs', 'partial_struts', 'boundary_hubs', 'partial_panes']:
    if separated[key]:
        parts_to_show.append(separated[key])
        part_colors.append(structure_color)
        part_alphas.append(1.0)
        part_names.append(NAMES[key])

# Add each pane individually with its random color
for i, (pane, color) in enumerate(zip(pane_solids, pane_colors)):
    parts_to_show.append(pane)
    part_colors.append(color)
    part_alphas.append(0.5)  # Semi-transparent
    part_names.append(f"Pane {i+1}")

print(f"Stained Glass View:")
print(f"  Structure elements: {sum(1 for k in ['complete_struts', 'complete_hubs', 'boundary_hubs'] if separated[k])}")
print(f"  Individual panes: {len(pane_solids)}")
show(*parts_to_show, colors=part_colors, alphas=part_alphas, names=part_names)


Stained Glass View:
  Structure elements: 3
  Individual panes: 73
cccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc


## Manufacturability Analysis

Analyzing unique part counts for manufacturing.

In [18]:
# Run Full Manufacturability Analysis
# This uses the analysis module to classify all parts as complete or cut

# Get parameters from the generation
vertices = info_prism['vertices']
edges = info_prism['edges']
faces = info_prism['faces']
radius_cm = info_prism['radius_cm']
portion = info_prism['portion']
strut_width = info_prism['strut_width']
hub_inset = info_prism['hub_inset']

# Run comprehensive analysis
analysis = full_manufacturability_analysis(
    vertices=vertices,
    edges=edges,
    faces=faces,
    radius_cm=radius_cm,
    portion=portion,
    strut_width=strut_width,
    hub_inset=hub_inset,
    length_tol_cm=0.1,
    angle_tol_deg=1.0,
    size_tol_cm=0.5,
    margin=0.2,
)

# Print the formatted report
print(format_analysis_report(analysis, units='in'))


MANUFACTURABILITY ANALYSIS REPORT

Dome Parameters:
  Radius: 8.0 ft (243.8 cm)
  Portion: 0.5
  Strut Width: 3.5" (8.9 cm)
  Hub Inset: 1.2" (3.1 cm)

--------------------------------------------------------------------------------
STRUTS
--------------------------------------------------------------------------------

Total Struts: 264
  Complete: 240 (5 unique lengths)
  Cut (at boundary): 24 (4 unique lengths)

Complete Struts by Length:
  14.6" (37.2 cm): 32 pieces
  16.3" (41.4 cm): 60 pieces
  16.7" (42.3 cm): 28 pieces
  16.8" (42.6 cm): 88 pieces
  17.5" (44.4 cm): 32 pieces

Cut Struts by Original Length:
  16.3" (41.4 cm): 4 pieces (trimmed at base)
  16.7" (42.3 cm): 8 pieces (trimmed at base)
  16.8" (42.6 cm): 8 pieces (trimmed at base)
  17.5" (44.4 cm): 4 pieces (trimmed at base)

--------------------------------------------------------------------------------
HUBS (all complete - none cut)
--------------------------------------------------------------------------------

In [19]:
# Detailed Parts Summary Table

print("=" * 90)
print("DETAILED PARTS SUMMARY - CHAPEL DOME (8ft, 3V)")
print("=" * 90)

struts = analysis['struts']
hubs = analysis['hubs']
windows = analysis['windows']
panes = analysis['panes']

# Create a nice summary table
print("\n{:<20} {:>12} {:>12} {:>12} {:>15}".format(
    "Part Type", "Complete", "Cut/Partial", "Total", "Unique Types"))
print("-" * 90)

print("{:<20} {:>12} {:>12} {:>12} {:>15}".format(
    "Struts",
    struts['complete']['count'],
    struts['cut']['count'],
    struts['total'],
    struts['complete']['unique_types'] + struts['cut']['unique_types']
))

# Hubs: all are complete pieces (none are cut)
print("{:<20} {:>12} {:>12} {:>12} {:>15}".format(
    "Hubs",
    hubs['total'],
    0,
    hubs['total'],
    hubs['interior']['unique_types'] + hubs['boundary']['unique_types']
))

print("{:<20} {:>12} {:>12} {:>12} {:>15}".format(
    "Windows (Openings)",
    windows['complete']['count'],
    windows['cut']['count'],
    windows['total'],
    windows['complete']['unique_types'] + windows['cut']['unique_types']
))

print("{:<20} {:>12} {:>12} {:>12} {:>15}".format(
    "Panes (Plates)",
    panes['complete']['count'],
    panes['cut']['count'],
    panes['total'],
    panes['complete']['unique_types'] + panes['cut']['unique_types']
))

print("-" * 90)

# Total linear feet of strut material
strut_total_cm = sum(
    s['length_cm'] for s in struts['complete']['struts']
) + sum(
    s['length_cm'] for s in struts['cut']['struts']
)
strut_total_ft = strut_total_cm / 30.48

print(f"\nTotal linear feet of strut material: {strut_total_ft:.0f} ft")

DETAILED PARTS SUMMARY - CHAPEL DOME (8ft, 3V)

Part Type                Complete  Cut/Partial        Total    Unique Types
------------------------------------------------------------------------------------------
Struts                        240           24          264              11
Hubs                          168            0          168              10
Windows (Openings)             73           16           89               8
Panes (Plates)                 73           16           89               8
------------------------------------------------------------------------------------------

Total linear feet of strut material: 363 ft


In [22]:
# Detailed Cut Lists for All Parts (with dual units)
from collections import Counter

def consolidate_lengths(by_length, tol_cm=0.3):
    """Consolidate lengths within tolerance (~1/8 inch)."""
    consolidated = {}
    for length_cm, count in by_length.items():
        key = round(length_cm / tol_cm) * tol_cm
        consolidated[key] = consolidated.get(key, 0) + count
    return consolidated

def consolidate_shapes(by_sig, tol_cm=0.5):
    """Consolidate shape signatures within tolerance."""
    consolidated = {}
    for sig, count in by_sig.items():
        num_sides = sig[0]
        size = round(sig[1] / tol_cm) * tol_cm
        key = (num_sides, size)
        consolidated[key] = consolidated.get(key, 0) + count
    return consolidated

def fmt_len(cm):
    """Format length as inches (cm)."""
    return f"{cm/2.54:.1f}\" ({cm:.1f} cm)"

def fmt_size(cm):
    """Format size as inches / cm."""
    return f"{cm/2.54:.1f}\" / {cm:.1f} cm"

print("=" * 90)
print("DETAILED CUT LISTS - ALL PARTS")
print("=" * 90)

# ============ STRUTS ============
print("\n" + "=" * 90)
print("STRUTS - Complete (Full Length)")
print("=" * 90)
print(f"\nCross-section: {strut_width/2.54:.2f}\" x {strut_width/2.54:.2f}\" ({strut_width:.1f} cm x {strut_width:.1f} cm)")
print(f"Cut type: Perpendicular (90°) at both ends")
print(f"\n{'Type':<6} {'Length':<25} {'Count':<8} {'Notes'}")
print("-" * 70)

complete_lengths = consolidate_lengths(struts['complete']['by_length'])
for i, (length_cm, count) in enumerate(sorted(complete_lengths.items()), 1):
    print(f"S{i:<5} {fmt_len(length_cm):<25} {count:<8} Full strut")

if struts['cut']['by_length']:
    print("\n" + "-" * 70)
    print("STRUTS - Cut at Boundary (Partial)")
    print("-" * 70)
    cut_lengths = consolidate_lengths(struts['cut']['by_length'])
    for i, (length_cm, count) in enumerate(sorted(cut_lengths.items()), 1):
        print(f"SC{i:<4} {fmt_len(length_cm):<25} {count:<8} Trimmed at base")

# ============ HUBS (all are complete pieces) ============
print("\n" + "=" * 90)
print("HUBS (all complete - none cut)")
print("=" * 90)
print(f"\n{'Type':<6} {'Config':<40} {'Count':<8}")
print("-" * 70)

# Combine interior and boundary hub counts by signature
all_hub_sigs = {}
for sig, count in hubs['interior']['by_signature'].items():
    all_hub_sigs[sig] = all_hub_sigs.get(sig, 0) + count
for sig, count in hubs['boundary']['by_signature'].items():
    all_hub_sigs[sig] = all_hub_sigs.get(sig, 0) + count

for i, (sig, count) in enumerate(sorted(all_hub_sigs.items()), 1):
    num_struts = sig[0]
    angles = sig[1]
    angles_str = ", ".join(f"{a:.0f}°" for a in angles[:3])
    if len(angles) > 3:
        angles_str += "..."
    config = f"{num_struts}-strut [{angles_str}]"
    print(f"H{i:<5} {config:<40} {count:<8}")

# ============ WINDOWS ============
print("\n" + "=" * 90)
print("WINDOWS (Frame Openings) - Complete")
print("=" * 90)
print(f"\n{'Type':<6} {'Shape':<10} {'Size (side-to-side)':<25} {'Count':<8}")
print("-" * 70)

complete_windows = consolidate_shapes(windows['complete']['by_signature'])
for i, ((num_sides, size_cm), count) in enumerate(sorted(complete_windows.items()), 1):
    shape = "Hex" if num_sides == 6 else "Pent" if num_sides == 5 else f"{num_sides}-gon"
    print(f"W{i:<5} {shape:<10} {fmt_size(size_cm):<25} {count:<8}")

if windows['cut']['by_signature']:
    print("\n" + "-" * 70)
    print("WINDOWS - Cut at Boundary (Partial)")
    print("-" * 70)
    cut_windows = consolidate_shapes(windows['cut']['by_signature'])
    for i, ((num_sides, size_cm), count) in enumerate(sorted(cut_windows.items()), 1):
        shape = "Hex" if num_sides == 6 else "Pent" if num_sides == 5 else f"{num_sides}-gon"
        print(f"WC{i:<4} {shape:<10} {fmt_size(size_cm):<25} {count:<8}")

# ============ PANES ============
print("\n" + "=" * 90)
print("PANES (Window Plates) - Complete")
print("=" * 90)
print(f"\n{'Type':<6} {'Shape':<10} {'Size (side-to-side)':<25} {'Count':<8}")
print("-" * 70)

complete_panes = consolidate_shapes(panes['complete']['by_signature'])
for i, ((num_sides, size_cm), count) in enumerate(sorted(complete_panes.items()), 1):
    shape = "Hex" if num_sides == 6 else "Pent" if num_sides == 5 else f"{num_sides}-gon"
    print(f"P{i:<5} {shape:<10} {fmt_size(size_cm):<25} {count:<8}")

if panes['cut']['by_signature']:
    print("\n" + "-" * 70)
    print("PANES - Cut at Boundary (Custom)")
    print("-" * 70)
    cut_panes = consolidate_shapes(panes['cut']['by_signature'])
    for i, ((num_sides, size_cm), count) in enumerate(sorted(cut_panes.items()), 1):
        shape = "Hex" if num_sides == 6 else "Pent" if num_sides == 5 else f"{num_sides}-gon"
        print(f"PC{i:<4} {shape:<10} {fmt_size(size_cm):<25} {count:<8}")

print("\n" + "=" * 90)

DETAILED CUT LISTS - ALL PARTS

STRUTS - Complete (Full Length)

Cross-section: 3.50" x 3.50" (8.9 cm x 8.9 cm)
Cut type: Perpendicular (90°) at both ends

Type   Length                    Count    Notes
----------------------------------------------------------------------
S1     14.6" (37.2 cm)           32       Full strut
S2     16.3" (41.4 cm)           60       Full strut
S3     16.7" (42.3 cm)           28       Full strut
S4     16.8" (42.6 cm)           88       Full strut
S5     17.5" (44.4 cm)           32       Full strut

----------------------------------------------------------------------
STRUTS - Cut at Boundary (Partial)
----------------------------------------------------------------------
SC1    16.3" (41.4 cm)           4        Trimmed at base
SC2    16.7" (42.3 cm)           8        Trimmed at base
SC3    16.8" (42.6 cm)           8        Trimmed at base
SC4    17.5" (44.4 cm)           4        Trimmed at base

HUBS (all complete - none cut)

Type   Config    

In [23]:
# Detailed Pane Shape Analysis
# Shows that panes are IRREGULAR polygons, not regular hexagons/pentagons

import math

print("=" * 90)
print("DETAILED PANE SHAPE ANALYSIS")
print("=" * 90)
print("\nNOTE: Panes are IRREGULAR polygons - each has unique edge lengths and angles.")
print("They are NOT interchangeable, even among 'hexagons' or 'pentagons'.")

def analyze_face_shape(face_verts):
    """Analyze a polygon's shape and return edge lengths and angles."""
    n = len(face_verts)
    
    # Calculate edge lengths
    edge_lengths = []
    for i in range(n):
        v1, v2 = face_verts[i], face_verts[(i+1) % n]
        length = math.sqrt((v2[0]-v1[0])**2 + (v2[1]-v1[1])**2 + (v2[2]-v1[2])**2)
        edge_lengths.append(length)
    
    return edge_lengths

# Group faces by number of sides and analyze
hex_faces = [(i, f) for i, f in enumerate(faces) if len(f) == 6]
pent_faces = [(i, f) for i, f in enumerate(faces) if len(f) == 5]

# Analyze hexagons
print("\n" + "-" * 90)
print(f"HEXAGONAL PANES ({len(hex_faces)} total)")
print("-" * 90)

# Collect all hexagon edge data
hex_edge_data = []
for face_idx, face in hex_faces:
    face_verts = [vertices[i] for i in face]
    edges = analyze_face_shape(face_verts)
    hex_edge_data.append({
        'face_idx': face_idx,
        'edges_in': [e/2.54 for e in edges],
        'edges_cm': edges,
        'min': min(edges),
        'max': max(edges),
        'variation': (max(edges) - min(edges)) / min(edges) * 100
    })

# Group by similar edge patterns (rounded)
def edge_signature(edges, tol=0.5):
    """Create a signature for edge pattern."""
    rounded = tuple(round(e / tol) * tol for e in sorted(edges))
    return rounded

hex_by_pattern = {}
for h in hex_edge_data:
    sig = edge_signature(h['edges_cm'])
    if sig not in hex_by_pattern:
        hex_by_pattern[sig] = []
    hex_by_pattern[sig].append(h)

print(f"\nUnique hexagon patterns: {len(hex_by_pattern)}")
print(f"\n{'Pattern':<6} {'Edge Lengths (inches)':<50} {'Count':<8} {'Variation'}")
print("-" * 90)

for i, (sig, items) in enumerate(sorted(hex_by_pattern.items(), key=lambda x: x[0]), 1):
    # Show representative edge lengths
    rep = items[0]
    edges_str = ", ".join(f"{e:.1f}" for e in rep['edges_in'])
    var = rep['variation']
    print(f"HEX-{i:<2} [{edges_str}]  {len(items):<8} {var:.1f}%")

# Show overall hex statistics
all_hex_edges = [e for h in hex_edge_data for e in h['edges_in']]
print(f"\nHexagon edge range: {min(all_hex_edges):.1f}\" to {max(all_hex_edges):.1f}\" ({(max(all_hex_edges)-min(all_hex_edges)):.1f}\" span)")

# Analyze pentagons
print("\n" + "-" * 90)
print(f"PENTAGONAL PANES ({len(pent_faces)} total)")
print("-" * 90)

pent_edge_data = []
for face_idx, face in pent_faces:
    face_verts = [vertices[i] for i in face]
    edges = analyze_face_shape(face_verts)
    pent_edge_data.append({
        'face_idx': face_idx,
        'edges_in': [e/2.54 for e in edges],
        'edges_cm': edges,
        'min': min(edges),
        'max': max(edges),
        'variation': (max(edges) - min(edges)) / min(edges) * 100
    })

pent_by_pattern = {}
for p in pent_edge_data:
    sig = edge_signature(p['edges_cm'])
    if sig not in pent_by_pattern:
        pent_by_pattern[sig] = []
    pent_by_pattern[sig].append(p)

print(f"\nUnique pentagon patterns: {len(pent_by_pattern)}")
print(f"\n{'Pattern':<6} {'Edge Lengths (inches)':<45} {'Count':<8} {'Variation'}")
print("-" * 90)

for i, (sig, items) in enumerate(sorted(pent_by_pattern.items(), key=lambda x: x[0]), 1):
    rep = items[0]
    edges_str = ", ".join(f"{e:.1f}" for e in rep['edges_in'])
    var = rep['variation']
    print(f"PENT-{i} [{edges_str}]  {len(items):<8} {var:.1f}%")

if pent_edge_data:
    all_pent_edges = [e for p in pent_edge_data for e in p['edges_in']]
    print(f"\nPentagon edge range: {min(all_pent_edges):.1f}\" to {max(all_pent_edges):.1f}\" ({(max(all_pent_edges)-min(all_pent_edges)):.1f}\" span)")

print("\n" + "=" * 90)
print("MANUFACTURING NOTE:")
print("  Each pane must be cut to its specific shape - they are NOT interchangeable.")
print("  Use window frame as template or generate individual pane drawings.")
print("=" * 90)


DETAILED PANE SHAPE ANALYSIS

NOTE: Panes are IRREGULAR polygons - each has unique edge lengths and angles.
They are NOT interchangeable, even among 'hexagons' or 'pentagons'.

------------------------------------------------------------------------------------------
HEXAGONAL PANES (81 total)
------------------------------------------------------------------------------------------

Unique hexagon patterns: 3

Pattern Edge Lengths (inches)                              Count    Variation
------------------------------------------------------------------------------------------
HEX-1  [16.6, 14.7, 16.8, 16.8, 16.8, 14.7]  32       14.4%
HEX-2  [16.8, 16.3, 16.3, 16.8, 16.3, 16.3]  17       3.3%
HEX-3  [16.8, 16.8, 16.3, 17.5, 17.5, 16.3]  32       7.4%

Hexagon edge range: 14.7" to 17.5" (2.8" span)

------------------------------------------------------------------------------------------
PENTAGONAL PANES (8 total)
-----------------------------------------------------------------------